# Init

In [1]:
from data.datasource.tv_data_feed_api import get_data
from tvDatafeed import TvDatafeed, Interval
import requests

import os
from datetime import datetime
from openpyxl import Workbook, load_workbook
import time
import pandas as pd
import matplotlib.pyplot as plt

import business.utils.trading_signals as ts
import business.utils.trading_indicators as ti
import data.datasource.binance_api as ba
LOG_FILE = r"F:\Desktop\git\crypto_bi\log.xlsx"

from dotenv import load_dotenv

load_dotenv()


# historical_data = get_data(symbol="BTCUSDT", exchange="BINANCE", interval=Interval.in_1_hour, n_bars=100000)
# historical_data

True

In [2]:
def send_notification(message):
    # Replace these with your actual Pushover API token and user key
    app_token = os.getenv('NOTIFICATION_TOKEN')  # e.g., "a1b2c3d4e5f6g7h8i9j0"
    user_key = os.getenv('NOTIFICATION_USER_KEY')    # e.g., "u1v2w3x4y5z6"

    # Prepare the payload with a loud sound ('siren' is one example)
    payload = {
        'token': app_token,
        'user': user_key,
        'message': message,
        'sound': 'siren',  # Change this to any sound of your choice
        'priority': 1      # priority 1 or 2 can trigger an alert mode on some devices
    }
    
    # Send the POST request to Pushover
    response = requests.post("https://api.pushover.net/1/messages.json", data=payload)
    
    if response.status_code == 200:
        print("Notification sent successfully!")
    else:
        print("Failed to send notification. Response:", response.text)


In [3]:
def log_to_excel(timestamp, symbol, status, message,notified_signal_value,check_func, log_file=LOG_FILE):
    # If the log file doesn't exist, create it with headers
    if not os.path.exists(log_file):
        wb = Workbook()
        ws = wb.active
        ws.title = "Log"
        # Write header row
        ws.append(["Timestamp", "Symbol", "Status", "Message", "Notified Signal Value", "Check Function"])
        wb.save(log_file)
    
    # Load the existing workbook and append a new row
    wb = load_workbook(log_file)
    ws = wb.active
    ws.append([timestamp, symbol, status, message,notified_signal_value,check_func])
    wb.save(log_file)


# Live Signal Multiple Stocks

In [4]:
# List of symbols to monitor
symbols = ["BTCUSDT", "ETHUSDT"]

check_signals = [ts.very_low_price,
                    ts.very_high_price
                    ]

In [ ]:
def run_checks_and_notify(price_threshold):
    for symbol in symbols:
        # Retrieve data for the symbol (adjust parameters as needed)
        df = ba.get_binance_klines(symbol=symbol, interval='1h', limit=1000)
        """
        get_data(
            symbol=symbol,
            exchange="BINANCE",
            interval=Interval.in_1_hour,
            n_bars=1000
        )
        """
        # Run each check function on the data
        for check_func in check_signals:
            log_df = pd.read_excel(LOG_FILE)
            last_valid_row = log_df[
                (log_df['Symbol'] == symbol) &
                (log_df['Check Function'] == check_func.__name__) &
                (log_df['Status'])
            ].tail(1)
            status, message, notified_signal_value = check_func(symbol, df, last_valid_row)
            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            log_to_excel(current_time, symbol, status, message,notified_signal_value,check_func.__name__)
            if status:
                send_notification(message)


In [6]:
run_checks_and_notify(price_threshold=2560)

In [12]:
data = ba.get_binance_klines(symbol='BTCUSDT', interval='1m', limit=1000)
df = pd.DataFrame(data, columns=[
    "timestamp", "open", "high", "low", "close", "volume",
    "close_time", "quote_asset_volume", "number_of_trades",
    "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"
])
df["timestamp"] = pd.to_datetime(df["timestamp"], unit='ms')

In [13]:
df

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
0,2025-02-24 08:16:00,95702.39000000,95702.39000000,95692.72000000,95692.73000000,3.05584000,1740385019999,292433.50604700,519,1.73163000,165709.57374830,0
1,2025-02-24 08:17:00,95692.73000000,95694.88000000,95656.44000000,95685.78000000,12.45247000,1740385079999,1191432.16646010,1681,4.91289000,470072.13849020,0
2,2025-02-24 08:18:00,95685.78000000,95685.78000000,95661.50000000,95661.50000000,3.24425000,1740385139999,310389.45726780,1068,0.94529000,90434.20718880,0
3,2025-02-24 08:19:00,95661.51000000,95661.51000000,95617.96000000,95617.97000000,5.38583000,1740385199999,515099.31728070,1509,0.59348000,56761.61848550,0
4,2025-02-24 08:20:00,95617.96000000,95617.97000000,95587.19000000,95593.43000000,38.49578000,1740385259999,3680171.81296100,2529,3.14339000,300491.39945970,0
...,...,...,...,...,...,...,...,...,...,...,...,...
995,2025-02-25 00:51:00,91992.20000000,92057.74000000,91934.95000000,92057.74000000,66.72444000,1740444719999,6138542.57791460,8951,48.16847000,4431389.71204610,0
996,2025-02-25 00:52:00,92057.74000000,92096.46000000,92000.00000000,92088.00000000,30.95302000,1740444779999,2848790.33368530,4796,23.34370000,2148372.07255090,0
997,2025-02-25 00:53:00,92088.00000000,92162.37000000,92075.52000000,92089.55000000,45.88494000,1740444839999,4226403.11323840,6185,29.60955000,2727194.93650960,0
998,2025-02-25 00:54:00,92089.56000000,92097.73000000,91923.64000000,91947.26000000,46.51301000,1740444899999,4278661.13299650,7083,18.77496000,1726493.05067920,0


In [17]:
log_df = pd.read_excel(LOG_FILE)
last_valid_row = log_df[
    (log_df['Symbol'] == 'ETHUSDT') &
    # (log_df['Check Function'] == '') &
    (log_df['Status'] == False)
].tail(1)

In [22]:
last_valid_row['Symbol'].values[0]

'ETHUSDT'